# MAST30034 Project 2

## Data Pre-processing

In [1]:
import pandas as pd
import requests
import numpy as np
import re
import os
import json
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from geopy import distance

/Users/kitty/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### External Data

#### 1. SA2 Shapefile

In [2]:
sf = gpd.read_file("../data/raw/SA2/SA2_2021_AUST_GDA2020.shp")
# only leave useful cols
SA2_inf = ['SA2_CODE21', 'SA2_NAME21', 'geometry']
sf = sf[SA2_inf]
sf.crs = 'EPSG: 4326'
#sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

#### 2. Population

In [3]:
df_popu = pd.read_excel('../data/raw/population.xlsx', sheet_name='Table 1')
# Delete the rows that has Nan in Unnamed 1 column
df_popu = df_popu.dropna(subset=["Unnamed: 1"]) 

# Change the name of columns to the first row of the dataframe
import numpy as np
# Get the first row data
array = np.array(df_popu)
list = array.tolist()
list = list[0]

# Change the column name to the first row data
df_popu.columns = list
# Delete the first row 
df_popu.drop([6], inplace=True)

# Reset index
df_popu.reset_index(drop=True, inplace=True)


df_popu = df_popu.drop(['S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code', 
                  'SA4 name', 'SA3 code', 'SA3 name'], axis=1)
# Change the name of each columns 
df_popu.columns = ['SA2 code','SA2 name', '2001', '2002', '2003', '2004' , 
                  '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', 
                  '2014', '2015', '2016','2017', '2018', '2019', '2020', '2021','NaN','change 01-21', 
                  'change %', 'NaN', 'km2', 'persons/km2']

# Only keep the useful columns 
df_popu = df_popu[['SA2 code', 'SA2 name', '2016', '2017', '2018', '2019', '2020', '2021']]
df_popu['SA2_code'] = df_popu['SA2 code']

In [4]:
popu_rate = []
for i in df_popu.index:
    if df_popu['2018'][i] != 0:
        popu_rate.append(float((df_popu['2021'][i]-df_popu['2018'][i])/df_popu['2018'][i]))
    else:
        popu_rate.append('NaN')     
df_popu['popu_rate18-21'] = np.array(popu_rate).tolist()
#df_popu['popu_rate18-21'] = pd.to_numeric(popu_rate['popu_rate18-21'])

popu2022 = []
popu2025 = []
for i in df_popu.index:
    if df_popu['popu_rate18-21'][i] == 'NaN':
        popu2022.append(df_popu['2019'][i])
    else:
        popu2022.append(df_popu['2019'][i] + df_popu['2019'][i]*float(df_popu['popu_rate18-21'][i]))
df_popu['popu2022'] = np.array(popu2022).tolist()
for i in df_popu.index:
    if df_popu['popu_rate18-21'][i] == 'NaN':
        popu2025.append(df_popu['popu2022'][i])
    else:
        popu2025.append(df_popu['popu2022'][i] + df_popu['popu2022'][i]*float(df_popu['popu_rate18-21'][i]))
df_popu['popu2025'] = np.array(popu2025).tolist()
lst = ['SA2_code', 'SA2 name', 'popu2022', 'popu2025']
df_popu = df_popu[lst]

#### 3. Income

In [5]:
df_income = pd.read_excel('../data/raw/income.xlsx', sheet_name='Table 1')

df_income = df_income.dropna(subset=["Unnamed: 1"]) 
df_income = df_income.iloc[3186:19235] # the income instance counted by SA2 code
att = ['            Australian Bureau of Statistics','Unnamed: 2', 'Unnamed: 7']
df_income = df_income[att]
df_income = df_income.reset_index(drop = True)
df_income.rename(columns = {'            Australian Bureau of Statistics':'SA2_code', 'Unnamed: 2':'Year', 
                            'Unnamed: 7':'Mean Income'}, inplace = True)

df_income_2014 = df_income.loc[df_income['Year'] == 2014]
df_income_2016 = df_income.loc[df_income['Year'] == 2016]
df_income_2017 = df_income.loc[df_income['Year'] == 2017]

df_income = pd.merge(df_income_2014, df_income_2016, on='SA2_code')
df_income = pd.merge(df_income, df_income_2017, on='SA2_code')

df_income.rename(columns = {'Mean Income_x':'Mean Income2014', 'Mean Income_y': 'Mean Income2016', 
                            'Mean Income':'Mean Income2017'}, inplace = True)

#### 4. PTV

In [6]:
stops_2 = pd.read_csv("../data/raw/ptv/2/stops.txt")
stops_2 = stops_2.loc[:,["stop_name","stop_lat","stop_lon"]]
stops_3 = pd.read_csv("../data/raw/ptv/3/stops.txt")
stops_3 = stops_3.loc[:,["stop_name","stop_lat","stop_lon"]]
stops_4 = pd.read_csv("../data/raw/ptv/4/stops.txt")
stops_4 = stops_4.loc[:,["stop_name","stop_lat","stop_lon"]]

gdf_stops_2 = gpd.GeoDataFrame(
    stops_2, geometry=gpd.points_from_xy(stops_2['stop_lon'], stops_2['stop_lat']))
gdf_stops_3 = gpd.GeoDataFrame(
    stops_3, geometry=gpd.points_from_xy(stops_3['stop_lon'], stops_3['stop_lat']))
gdf_stops_4 = gpd.GeoDataFrame(
    stops_4, geometry=gpd.points_from_xy(stops_4['stop_lon'], stops_4['stop_lat']))

gdf_stops_2.crs = 'EPSG: 4326'
gdf_stops_3.crs = 'EPSG: 4326'
gdf_stops_4.crs = 'EPSG: 4326'
stops2_sa2 = gdf_stops_2.sjoin(sf, how='inner', predicate='within')
stops3_sa2 = gdf_stops_3.sjoin(sf, how='inner', predicate='within')
stops4_sa2 = gdf_stops_4.sjoin(sf, how='inner', predicate='within')

geoJSON_2 = stops2_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()
geoJSON_3 = stops3_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()
geoJSON_4 = stops4_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()

stops2_df = stops2_sa2.groupby('SA2_CODE21')['geometry'].count()
stops2_df = stops2_df.to_frame().reset_index()
stops2_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops3_df = stops3_sa2.groupby('SA2_CODE21')['geometry'].count()
stops3_df = stops3_df.to_frame().reset_index()
stops3_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops4_df = stops4_sa2.groupby('SA2_CODE21')['geometry'].count()
stops4_df = stops4_df.to_frame().reset_index()
stops4_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops2_sa2 = pd.merge(left=stops2_sa2, right=stops2_df, left_on='SA2_CODE21', right_on='SA2_code')

att = ['SA2_code', 'stop_count']
stops2_sa2_count = stops2_sa2[att]

#### 5. School

In [7]:
df_school = pd.read_csv('../data/raw/school2021.csv')

# Filter the columns and rename them.
df_school = df_school[['X', 'Y', 'Address_Line_1', 'School_Name']]  # type: ignore
df_school.rename(columns={'X':'school_X', 'Y':'school_Y',
                          'Address_Line_1':'school_addr', 'School_Name': 'school_name'}, inplace = True)

# now change our school dataframe to geodataframe by transforming the x, y coordinates to point geometry.
gdf_school = gpd.GeoDataFrame(
    df_school, geometry=gpd.points_from_xy(df_school['school_X'], df_school['school_Y']))

gdf_school.crs = 'EPSG: 4326'

# Then apply spatial join
school_sa2 = gdf_school.sjoin(sf, how='inner', predicate='within')

school_df = school_sa2.groupby('SA2_CODE21')['geometry'].count()
school_df = school_df.to_frame().reset_index()
school_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'school_count'}, inplace = True)

#### 6. Hospital

In [8]:
df_hospital = pd.read_csv('../data/raw/hospital2021.csv')
# Filter the columns and rename them.
df_hospital = df_hospital[['X','Y','addr_street','name']]
df_hospital.rename(columns={'X':'hospital_X', 'Y':'hospital_Y', 'addr_street':'hospital_addr','name':'hospital_name'}, inplace = True)

# Remove the null value.
df_hospital = df_hospital.dropna(axis=0, subset=['hospital_X', 'hospital_Y'])
# now change our hospital dataframe to geodataframe by transforming the x, y coordinates to point geometry.
gdf_hospital = gpd.GeoDataFrame(
    df_hospital, geometry=gpd.points_from_xy(df_hospital['hospital_X'], df_hospital['hospital_Y']))

gdf_hospital.crs = 'EPSG: 4326'

# Then apply spatial join
hospital_sa2 = gdf_hospital.sjoin(sf, how='inner', predicate='within')

hospital_df = hospital_sa2.groupby('SA2_CODE21')['geometry'].count()
hospital_df = hospital_df.to_frame().reset_index()
hospital_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'hos_count'}, inplace = True)

### Domain Rental Prices

In [9]:
df_domain = pd.read_json('../data/raw/domain1.json')

df_domain = df_domain.T
df_domain = df_domain.drop('desc', axis = 1)
df_domain = df_domain[df_domain['rooms'].astype(bool)]
df_domain = df_domain.reset_index(drop = True)

bed = []
bath = []
parking = []
Longitude = []
Latitude = []
postcode = []

for i in df_domain.index:
    
    postcode.append(df_domain['name'][i][-4:])
    
    for r in ((',', ''), ('$', ''), ('.00', '')):
        df_domain['cost_text'][i] = df_domain['cost_text'][i].replace(*r)
    # convert room col: [bed, bath, parking] to individual col
    bed.append(df_domain['rooms'][i][0][0])
    bath.append(df_domain['rooms'][i][1][0]) 
    if len(df_domain['rooms'][i]) == 3:    # if no parking info, default no parkings      
        parking.append(df_domain['rooms'][i][2][0])
    else:     
        parking.append('0')
    Longitude.append(df_domain['coordinates'][i][0])
    Latitude.append(df_domain['coordinates'][i][1])

df_domain['Postcode'] = np.array(postcode).tolist()
df_domain['Prices'] = df_domain['cost_text'].str.extract('(\d+)') 
df_domain['Prices'] = pd.to_numeric(df_domain['Prices'])
df_domain['Bedrooms'] = np.array(bed).tolist()
df_domain['Bathrooms'] = np.array(bath).tolist()
df_domain['Parkings'] = np.array(parking).tolist()
df_domain['Longitude'] = Longitude
df_domain['Latitude'] = Latitude

df_domain = df_domain.drop('rooms', axis = 1)
df_domain = df_domain.drop('cost_text', axis = 1)
df_domain = df_domain.dropna().reset_index(drop = True)
df_domain = df_domain.drop('coordinates', axis = 1)

# remove extreme prices
df_domain = df_domain[(df_domain['Prices'] > 100) & (df_domain['Prices'] < 10000)]
# remove duplicate
df_domain = df_domain.drop_duplicates()
df_domain = df_domain.reset_index(drop = True)

### Data Aggregation

In [10]:
# build geodataframe
gpd_domain = gpd.GeoDataFrame(
    df_domain, geometry=gpd.points_from_xy(df_domain.Latitude, df_domain.Longitude))

# standardlize crs
gpd_domain.crs = "EPSG:4326"

gpd_domain = gpd_domain.sjoin(sf, how="inner", predicate='within')
gpd_domain = gpd_domain.reset_index(drop = True)

gpd_domain['SA2_code'] = gpd_domain['SA2_CODE21']
gpd_domain = gpd_domain.drop('SA2_CODE21', axis = 1)

# merge population by SA2
df_popu['SA2_code'] = df_popu['SA2_code'].apply(str)
gpd_domain = pd.merge(gpd_domain, df_popu, on='SA2_code')

# merge income by SA2
df_income['SA2_code'] = df_income['SA2_code'].apply(str)
gpd_domain = pd.merge(gpd_domain, df_income, on='SA2_code')
inco_rate = []
for i in gpd_domain.index:
    inco_rate.append(float((gpd_domain['Mean Income2017'][i] - gpd_domain['Mean Income2014'][i])/
                           gpd_domain['Mean Income2014'][i]))
gpd_domain['Income rate14-17'] = np.array(inco_rate).tolist()
inco2019 = []
inco2022 = []
inco2025 = []
for i in gpd_domain.index:
    inco2019.append(gpd_domain['Mean Income2016'][i] + gpd_domain['Mean Income2016'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2019'] = np.array(inco2019).tolist()
for i in gpd_domain.index:
    inco2022.append(gpd_domain['Mean Income2019'][i] + gpd_domain['Mean Income2019'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2022'] = np.array(inco2022).tolist()
for i in gpd_domain.index:
    inco2025.append(gpd_domain['Mean Income2022'][i] + gpd_domain['Mean Income2022'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2025'] = np.array(inco2025).tolist()

In [11]:
#gpd_domain = pd.merge(gpd_domain, stops2_sa2_count, on='SA2_code')
#gpd_domain = pd.merge(gpd_domain, hospital_df, on='SA2_code')
#gpd_domain = pd.merge(gpd_domain, school_df, on='SA2_code')

In [12]:
att = ['name', 'Postcode', 'Prices', 'Bedrooms', 'Bathrooms', 'Parkings', 'geometry', 'SA2_code', 
       'Longitude', 'Latitude', 'popu2022', 'popu2025', 'Mean Income2022', 
      'Mean Income2025'] #'school_count', 'stop_count', 'hos_count'
gpd_domain = gpd_domain[att]

In [57]:
gpd_domain

,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025
0,1/62 Simpson St East Melbourne VIC 3002,3002,395.0,1,1,0,POINT (144.98911 -37.81067),206041119,-37.810666,144.989108,4965.669440,4555.306297,98664.943495,102957.654101
1,29/8 Wellington Crescent East Melbourne VIC 3002,3002,750.0,2,2,2,POINT (144.97767 -37.81722),206041119,-37.817224,144.977670,4965.669440,4555.306297,98664.943495,102957.654101
2,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101
3,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.669440,4555.306297,98664.943495,102957.654101
4,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.669440,4555.306297,98664.943495,102957.654101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,43 Bellman Avenue Clyde VIC 3978,3978,450.0,3,2,2,POINT (145.34012 -38.12376),212031303,-38.123765,145.340118,21129.675792,34771.277186,71382.469390,79738.921375
4032,11 Bonfire Way Cranbourne South VIC 3977,3977,450.0,3,2,1,POINT (145.23057 -38.13111),212031304,-38.131111,145.230571,23184.689777,27207.057756,59706.852893,65347.627829
4033,2 Creekview Way Cranbourne West VIC 3977,3977,500.0,4,2,2,POINT (145.25386 -38.10221),212031304,-38.102212,145.253859,23184.689777,27207.057756,59706.852893,65347.627829
4034,214 Evans Road Cranbourne West VIC 3977,3977,400.0,3,2,2,POINT (145.25655 -38.09045),212031304,-38.090450,145.256545,23184.689777,27207.057756,59706.852893,65347.627829


In [56]:
'''
df = gpd_domain.groupby('Postcode').SA2_code.nunique().to_frame().reset_index()
df1 = df[df['SA2_code']!=1]
mask = gpd_domain['Postcode'].isin(df1['Postcode'].values.tolist())
gpd_domain[mask]
'''

,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025
20,1310/450 St Kilda Road Melbourne VIC 3004,3004,620.0,2,2,1,POINT (144.97558 -37.83877),206051128,-37.838772,144.975576,16359.368002,15677.401525,116186.535199,127027.017333
21,19/32 Queens Road Melbourne VIC 3004,3004,525.0,2,1,1,POINT (144.97626 -37.84338),206051128,-37.843384,144.976264,16359.368002,15677.401525,116186.535199,127027.017333
22,507/23 Queens Road Melbourne VIC 3004,3004,575.0,2,1,1,POINT (144.97551 -37.84113),206051128,-37.841132,144.975511,16359.368002,15677.401525,116186.535199,127027.017333
23,104/12 Queens Road Melbourne VIC 3004,3004,440.0,1,1,1,POINT (144.97471 -37.83842),206051128,-37.838419,144.974713,16359.368002,15677.401525,116186.535199,127027.017333
24,1312/35 Albert Rd Melbourne VIC 3004,3004,570.0,2,2,1,POINT (144.97211 -37.83469),206051128,-37.834695,144.972112,16359.368002,15677.401525,116186.535199,127027.017333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,28 Saddleback Road Botanic Ridge VIC 3977,3977,470.0,4,3,2,POINT (145.29136 -38.14873),212031303,-38.148728,145.291357,21129.675792,34771.277186,71382.469390,79738.921375
4032,11 Bonfire Way Cranbourne South VIC 3977,3977,450.0,3,2,1,POINT (145.23057 -38.13111),212031304,-38.131111,145.230571,23184.689777,27207.057756,59706.852893,65347.627829
4033,2 Creekview Way Cranbourne West VIC 3977,3977,500.0,4,2,2,POINT (145.25386 -38.10221),212031304,-38.102212,145.253859,23184.689777,27207.057756,59706.852893,65347.627829
4034,214 Evans Road Cranbourne West VIC 3977,3977,400.0,3,2,2,POINT (145.25655 -38.09045),212031304,-38.090450,145.256545,23184.689777,27207.057756,59706.852893,65347.627829


## Distance

In [136]:
import openrouteservice as ors
import folium
import time

In [137]:
ors_key = "5b3ce3597851110001cf624852ff1ab2e1224e1d9d57e2ad0f541f1a"
client = ors.Client(key=ors_key)

In [138]:
distance = []

a = 0

while a < int(len(gpd_domain)/20):
    k = a*20
    for i, row in gpd_domain.iloc[k:k+20].iterrows():
        x = row['geometry'].x
        y = row['geometry'].y
        coor = [[x, y],[144.9628,-37.8102]]
        route = client.directions(coordinates=coor, profile='driving-car',format='geojson')
        dis = route['features'][0]['properties']['segments'][0]['distance']*0.000621371*1.60934
        distance.append(dis)
    time.sleep(30)

ApiError: 403 ({'error': 'Quota exceeded'})

In [141]:
distance 

[2.6381926266003477,
 2.127094055053294,
 2.3148935301785865,
 1.993894427328646,
 1.861394797647596,
 2.112394096137736,
 1.497395814976636,
 2.796792183335552,
 2.3148935301785865,
 2.17249392816665,
 2.234793754046872,
 2.048694274170318,
 2.2762936380601824,
 3.287590811618264,
 1.7218951875305661,
 2.1990938538233737,
 2.2832936184961623,
 2.904691881770158,
 2.1844938946283303,
 1.7218951875305661,
 2.6381926266003477,
 2.127094055053294,
 2.3148935301785865,
 1.993894427328646,
 1.861394797647596,
 2.112394096137736,
 1.497395814976636,
 2.796792183335552,
 2.3148935301785865,
 2.17249392816665,
 2.234793754046872,
 2.048694274170318,
 2.2762936380601824,
 3.287590811618264,
 1.7218951875305661,
 2.1990938538233737,
 2.2832936184961623,
 2.904691881770158,
 2.1844938946283303,
 1.7218951875305661,
 2.6381926266003477,
 2.127094055053294,
 2.3148935301785865,
 1.993894427328646,
 1.861394797647596,
 2.112394096137736,
 1.497395814976636,
 2.796792183335552,
 2.3148935301785865,
 

In [140]:
len(distance)

910

In [144]:
import csv

In [145]:
with open('../data/distance_list', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(distance)